In [ ]:
!pip install --upgrade kaggle



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d mariaherrerot/messidor2preprocess

In [ ]:
! kaggle competitions download aptos2019-blindness-detection

In [ ]:
! unzip aptos2019-blindness-detection.zip

In [ ]:
! unzip messidor2preprocess.zip

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
aptos_train = pd.read_csv('/content/train.csv')
aptos_test = pd.read_csv('/content/test.csv')

In [ ]:
aptos_train.info()

In [ ]:
idrid_train = pd.read_csv('/content/drive/MyDrive/ML/capestone/B. Disease Grading/2. Groundtruths/training.csv')
idrid_test = pd.read_csv('/content/drive/MyDrive/ML/capestone/B. Disease Grading/2. Groundtruths/test.csv')

In [ ]:
messidor_df = pd.read_csv("/content/messidor_data.csv")

In [ ]:
aptos_freq = aptos_train['diagnosis'].value_counts()

In [ ]:
aptos_freq

In [ ]:
idrid_train['Retinopathy grade'].value_counts()

In [ ]:
messidor_df['diagnosis'].value_counts()

In [ ]:
import cv2


In [ ]:
img = cv2.imread('/content/train_images/000c1434d8d7.png')

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.imshow(img)

In [ ]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(gray_img)

In [ ]:
# Apply Otsu's thresholding
_, otsu_threshold = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [ ]:
plt.imshow(otsu_threshold,cmap='gray')

In [ ]:
# Convert the binary mask to uint8 type
otsu_threshold_uint8 = otsu_threshold.astype('uint8')

In [ ]:
# Find contours in the binary mask
contours, _ = cv2.findContours(otsu_threshold_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# Sort contours based on their area in descending order
contours_sorted = sorted(contours, key=cv2.contourArea)

In [ ]:
# Create a copy of the original image
img_with_roi = img.copy()

In [ ]:
largest_contour = contours_sorted[0]
x, y, w, h = cv2.boundingRect(largest_contour)

In [ ]:
# Crop the region of interest (ROI) from the original image
roi = img[y:y+h, x:x+w]

In [ ]:
plt.imshow(roi, cmap='gray')
plt.title('Extracted ROI')

In [ ]:
_, binary_image = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY)


In [ ]:
plt.imshow(binary_image)

In [ ]:
kernel_size = (10, 10)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
opened_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)


In [ ]:
plt.imshow(opened_image)


In [ ]:
opened_image.shape

In [ ]:
roi.shape

In [ ]:
opened_image = opened_image.astype('uint8')



In [ ]:
gray_image = cv2.cvtColor(opened_image, cv2.COLOR_BGR2GRAY)


In [ ]:
plt.imshow(opened_image)

In [ ]:
result_image = cv2.bitwise_and(roi, opened_image)

In [ ]:
plt.imshow(result_image)

In [ ]:
denoised_image = cv2.fastNlMeansDenoisingColored(result_image, None, 1, 1, 7, 21)


In [ ]:
plt.imshow(denoised_image)

In [ ]:
yuv_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2YUV)

In [ ]:
plt.imshow(yuv_image)

In [ ]:
y_channel = yuv_image[:, :, 0]

In [ ]:
plt.imshow(y_channel)

In [ ]:
clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8, 8))
clahe_output = clahe.apply(y_channel)

In [ ]:
plt.imshow(clahe_output)

In [ ]:
enhanced_yuv_image = cv2.merge([clahe_output, yuv_image[:, :, 1], yuv_image[:, :, 2]])

In [ ]:
plt.imshow(enhanced_yuv_image)

In [ ]:
# Step 5: Convert the enhanced image back to BGR color space
enhanced_bgr_image = cv2.cvtColor(enhanced_yuv_image, cv2.COLOR_YUV2BGR)

In [ ]:
plt.imshow(enhanced_bgr_image)

In [ ]:
arr = [[0,0,0],[0,255,0],[1,1,1]]

In [ ]:
aptos_train['diagnosis'].value_counts()

In [ ]:
messidor_df['diagnosis'].value_counts()

In [ ]:
idrid_train['Retinopathy grade'].value_counts()

In [ ]:
aptos_train.head(8)

In [ ]:
aptos_train['diagnosis'].value_counts()

In [ ]:
aptos_subset_dataframe = aptos_train.head(300)

In [ ]:
aptos_subset_dataframe['diagnosis'].value_counts()

In [ ]:
idrid_subset_dataframe = idrid_train.head(300)

In [ ]:
idrid_subset_dataframe['Retinopathy grade'].value_counts()

In [ ]:
idrid_train['Retinopathy grade'].value_counts()

In [ ]:
messidor_subset_dataframe = messidor_df.head(300)

In [ ]:
messidor_subset_dataframe['diagnosis'].value_counts()

In [ ]:
messidor_df['diagnosis'].value_counts()

In [ ]:
import os

In [ ]:
aptos_data = pd.DataFrame(columns=['id_code', 'image','label'])

In [ ]:
aptos_data.head()

In [ ]:
aptos_train_path = '/content/train_images'

In [ ]:
aptos_data.info()

In [ ]:
for index, row in aptos_subset_dataframe.iterrows():
    # Extract id_code and diagnosis from the DataFrame
    id_code = row['id_code']
    diagnosis = row['diagnosis']
    image_path = os.path.join(aptos_train_path, f"{id_code}.png")
    img = cv2.imread(image_path)
    aptos_data=aptos_data.append({'id_code': 1, 'image': img, 'label':diagnosis}, ignore_index=True)


In [ ]:
aptos_data.shape

In [ ]:
# idrid_data = pd.DataFrame(columns=['id_code', 'image','label'])

In [ ]:
# idrid_subset_dataframe.head(5)

In [ ]:
# idrid_train_path = '/content/drive/MyDrive/ML/capestone/B. Disease Grading/1. Original Images/a. Training Set/'

In [ ]:
# for index, row in idrid_subset_dataframe.iterrows():
#     # Extract id_code and diagnosis from the DataFrame
#     id_code = row['Image name']
#     diagnosis = row['Retinopathy grade']
#     image_path = os.path.join(idrid_train_path, f"{id_code}.jpg")
#     img = cv2.imread(image_path)
#     idrid_data=idrid_data.append({'id_code': 1, 'image': img, 'label':diagnosis}, ignore_index=True)

In [ ]:
aptos_data['label'].value_counts()

In [ ]:
def preprocess(img):
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Apply Otsu's thresholding
  _, otsu_threshold = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  # Convert the binary mask to uint8 type
  otsu_threshold_uint8 = otsu_threshold.astype('uint8')
  # Find contours in the binary mask
  contours, _ = cv2.findContours(otsu_threshold_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # Sort contours based on their area in descending order
  contours_sorted = sorted(contours, key=cv2.contourArea)
  largest_contour = contours_sorted[0]
  x, y, w, h = cv2.boundingRect(largest_contour)
  # Crop the region of interest (ROI) from the original image
  roi = img[y:y+h, x:x+w]
  _, binary_image = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY)
  kernel_size = (10, 10)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
  opened_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)
  opened_image = opened_image.astype('uint8')
  gray_image = cv2.cvtColor(opened_image, cv2.COLOR_BGR2GRAY)
  result_image = cv2.bitwise_and(roi, opened_image)
  denoised_image = cv2.fastNlMeansDenoisingColored(result_image, None, 1, 1, 7, 21)
  yuv_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2YUV)
  y_channel = yuv_image[:, :, 0]
  clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8, 8))
  clahe_output = clahe.apply(y_channel)
  enhanced_yuv_image = cv2.merge([clahe_output, yuv_image[:, :, 1], yuv_image[:, :, 2]])
  # Step 5: Convert the enhanced image back to BGR color space
  enhanced_bgr_image = cv2.cvtColor(enhanced_yuv_image, cv2.COLOR_YUV2BGR)
  resized_image = cv2.resize(enhanced_bgr_image, (512, 512))
  return resized_image




In [ ]:
img = cv2.imread('/content/train_images/000c1434d8d7.png')

In [ ]:
plt.imshow(img)

In [ ]:
img = preprocess(img)

In [ ]:
plt.imshow(img)

In [ ]:
aptos_data = aptos_data.drop(columns='id_code')


In [ ]:
aptos_data.head(2)

In [ ]:
image=  aptos_data['image'].iloc[0]

In [ ]:
plt.imshow(image)

In [ ]:
aptos_data['image'].iloc[0].shape

In [ ]:
for index, row in aptos_data.iterrows():
    img =  row['image']
    # Preprocess the image
    print("processing img",index)
    processed_image = preprocess(img)
    print("done")
    # Update the 'image' column with the processed image
    aptos_data.at[index, 'image'] = processed_image

In [ ]:
aptos_data['label'].value_counts()

In [ ]:

def brightness_altering(image, factor):
    return cv2.addWeighted(image, factor, np.zeros_like(image), 0, 0)

def contrast_altering(image, factor):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    return cv2.addWeighted(image, factor, gray, 1 - factor, 0)

def color_altering(image, factor):
    return cv2.addWeighted(image, factor, np.zeros_like(image), 0, 0)

def sharpness_altering(image, factor):
    blurred = cv2.GaussianBlur(image, (0, 0), factor)
    return cv2.addWeighted(image, 1.5, blurred, -0.5, 0)

In [ ]:
def apply_augmentation(img, factor, augmentation_type):
    if augmentation_type == 'brightness':
        return brightness_altering(img, factor)
    elif augmentation_type == 'contrast':
        return contrast_altering(img, factor)
    elif augmentation_type == 'color':
        return color_altering(img, factor)
    elif augmentation_type == 'sharpness':
        return sharpness_altering(img, factor)
    else:
        return img

In [ ]:
import random

In [ ]:
aptos_data.head(1)

In [ ]:
# Iterate over the dataset
for index, row in aptos_data.iterrows():
    print("img",index)
    label = row['label']
    img = row['image']

    # Define the number of additional copies based on the label
    num_copies = {
        0: 0,
        1: 5,
        2: 1,
        3: 7,
        4: 5
    }.get(label, 0)

    # Apply photometric augmentation and create additional copies
    for _ in range(num_copies):
        # Choose the type of augmentation based on your conditions
       augmentation_type = random.choice(['brightness', 'contrast', 'color', 'sharpness'])
       if augmentation_type:
            augmented_img = apply_augmentation(img, 1.8, augmentation_type)
            new_row = {'label': label, 'image': augmented_img}
            aptos_data = aptos_data.append(new_row, ignore_index=True)




In [ ]:
aptos_data['label'].value_counts()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:

# Define the model
model = Sequential()

# Block 1
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Block 2
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Block 3
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer
model.add(Flatten())

# Dense layer with dropout
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Classification dense layer
model.add(Dense(5, activation='softmax'))

lr = 0.0001  # Set your desired learning rate
optimizer = Adam(learning_rate=lr)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# Display the model summary
model.summary()


In [ ]:
X_train = aptos_data['image']

In [ ]:
X_train_resized = [cv2.resize(img, (224, 224)) for img in X_train]

In [ ]:
X_train_resized = np.array(X_train_resized)

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = aptos_data['label']

In [ ]:
y_train_encoded = to_categorical(y_train, num_classes=5)

In [ ]:
y_train_encoded[0]
type(y_train_encoded)

In [ ]:
# Train the model
history = model.fit(X_train_resized, y_train_encoded, epochs=100, batch_size=64)